# 验证区块

当获胜“矿工”将其打包的区块广播到网络中后，接收到区块的节点会独立验证每个区块的有效性。

In [1]:
# peer：节点对象，block：待验证区块
def verify_winner_block(peer, block):
    # 判断区块类型是否为Block
    if not isinstance(block, Block):
        return False

    # 验证工作量证明nonce
    if int(block.hash, 16) > caculate_target(block.bits):
        logger.info('{0} wrong answer'.format(block))
        return False

    # 获取区块交易
    txs = block.txs
    # 判断交易列表是否为list或tuple类型
    if not isinstance(txs, list) and \
            not isinstance(txs, tuple):
        logger.info('incorrect txs type in {0}'.format(block))
        return False

    # 判断交易的数量是否大于2
    if len(txs) < 2:
        logger.info('no enough txs for txs {0}'.format(block))
        return False

    # 获取非创币交易列表
    block_txs = txs[1:]
    #计算奖励总额，为“挖矿”奖励和交易费之和
    rewards = peer.get_block_reward() + peer.calculate_fees(block_txs)
    # 验证创币交易的有效性
    if not verify_coinbase(block.txs[0], rewards):
        logger.info('{0} coinbase incorrect'.format(block))
        return False

    # 验证区块交易列表中是否存在双重支付
    if double_payment_in_block_txs(block_txs):
        logger.info('double payment in {0}'.format(block))
        return False

    # 验证区块中非创币交易的有效性
    for tx in block_txs:
        if not verify_tx(peer, tx):
            return False
    return True

验证区块交易列表总是否存在双重支付是指，判断是否存在至少两条交易的输入单元使用了相同的UTXO。

In [2]:
# txs：非创币交易列表
def double_payment_in_block_txs(txs):
    # 将所有交易输入单元的定位指针存储到集合a中
    a = {vin.to_spend for tx in txs for vin in tx.tx_in}
    # 将所有交易输入单元的定位指针存储到列表b中
    b = [vin.to_spend for tx in txs for vin in tx.tx_in]
    
    # 如果a,b元素数量不相等，则为双重支付
    # 原因是集合中不含重复元素，列表则不是
    # 如果存在输入单元使用相同的UTXO，则集合的长度将比列表小
    return len(a) != len(b)

In [3]:
from simchain import Network
from simchain.peer import verify_winner_block
from simchain.consensus import mine
net = Network()
net.make_random_transactions()
zhangsan, lisi = net.peers[0], net.peers[1]
# 张三创建候选区块
zhangsan.create_candidate_block()
block = zhangsan.candidate_block
block

2019-06-14 00:10:04,465 - A blockchain p2p network created,12 peers joined
2019-06-14 00:10:04,510 - genesis block has been generated
2019-06-14 00:10:04,555 - peer(9, 99)(pid=5) created a transaction
2019-06-14 00:10:04,556 - peer(9, 99)(pid=5) sent a transaction to network
2019-06-14 00:10:05,416 - peer(9, 99)(pid=5)'s transaction verified by 11 peers
2019-06-14 00:10:05,458 - peer(41, 51)(pid=7) created a transaction
2019-06-14 00:10:05,460 - peer(41, 51)(pid=7) sent a transaction to network
2019-06-14 00:10:06,316 - peer(41, 51)(pid=7)'s transaction verified by 11 peers
2019-06-14 00:10:06,359 - peer(41, 51)(pid=7) created a transaction
2019-06-14 00:10:06,361 - peer(41, 51)(pid=7) sent a transaction to network
2019-06-14 00:10:07,224 - peer(41, 51)(pid=7)'s transaction verified by 11 peers
2019-06-14 00:10:07,267 - peer(7, 86)(pid=2) created a transaction
2019-06-14 00:10:07,269 - peer(7, 86)(pid=2) sent a transaction to network
2019-06-14 00:10:08,117 - peer(7, 86)(pid=2)'s trans

Block(hash:20616d1a002bba39688fbb3a4b0a0c2d29963b355d9c83c35af3800faaf6cf50)

In [4]:
# “挖矿”求nonce
nonce = mine(block)
nonce

358269

In [5]:
# 打包区块
block = block._replace(nonce=nonce)
block

Block(hash:000016d2c2678f7b41958dd019fb9dbbae4c440bb143c30531d32ca386631ead)

In [6]:
# 李四验证区块有效性
verify_winner_block(lisi, block)

True

In [7]:
# 获取交易的版本号
block.version

0

In [8]:
from simchain import Block
version = 2
prev_block_hash = block.prev_block_hash
timestamp = block.timestamp
bits = block.bits
nonce = block.nonce
txs = block.txs
# 新的区块仅改变了版本号
new_block = Block(version, prev_block_hash, timestamp, bits, nonce, txs)
new_block

Block(hash:1d82a2cfc2f9893e026c66ffe9aa68968f7310bcf0b2f63c3a2d99c1f7c04963)

In [9]:
# 新区块验证不通过，提示：答案不正确
verify_winner_block(lisi, new_block)

2019-06-14 00:10:17,574 - Block(hash:1d82a2cfc2f9893e026c66ffe9aa68968f7310bcf0b2f63c3a2d99c1f7c04963) wrong answer


False